In [31]:
inp = open('day10.txt', 'r').read()

In [32]:
machines = []
for line in inp.splitlines():
    parts = line.split()
    diagram = parts[0]
    joltages = list(map(int, parts[-1][1:-1].split(',')))
    buttons = []
    for button in parts[1:-1]:
        buttons_int = 0
        for b in button[1:-1].split(','):
            buttons_int |= 1 << int(b)
        buttons.append(buttons_int)
    target = int(diagram[1:-1].replace('.', '0').replace('#', '1')[::-1], 2)
    machines.append((target, buttons, joltages))

from collections import deque
def configure(machine):
    target, buttons, _ = machine
    
    queue = deque([(0, 0)])
    seen = set()
    while queue:
        cur, steps = queue.popleft()
        if cur == target:
            return steps
        for button in buttons:
            nxt = cur ^ button
            if nxt not in seen:
                seen.add(nxt)
                queue.append((nxt, steps + 1))

sum(configure(machine) for machine in machines)

514

In [33]:
import numpy as np
from scipy.optimize import milp, LinearConstraint, Bounds

machines = []
for line in inp.splitlines():
    parts = line.split()
    joltages = np.array(list(map(int, parts[-1][1:-1].split(','))))
    buttons = []
    for button in parts[1:-1]:
        button_list = [0] * len(joltages)
        for b in button[1:-1].split(','):
            button_list[int(b)] = 1
        buttons.append(button_list)
    machines.append((np.array(buttons), joltages))

import heapq
def configure2(machine):
    buttons, joltages = machine
    n = len(buttons)
    c = np.ones(n)
    A = buttons.T
    b = joltages
    bounds = Bounds(0, max(joltages))
    res = milp(c, constraints=LinearConstraint(A, b, b), bounds=bounds, integrality=np.ones(n))
    return int(res.fun)

from tqdm import tqdm
sum(configure2(machine) for machine in tqdm(machines))





100%|██████████| 197/197 [00:00<00:00, 374.24it/s]


21824